Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under he License.
the License.

## Imports and Parameters

In [ ]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time
import pandas as pd
import numpy as np

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model

In [ ]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q22'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ22ANCES'
CODE_TABLE = 'dbo.AQ22ANCES_Codes'
CLOSED_TABLE = 'dbo.Q22ANCESMultiResponse'
RESULTS_TABLE = 'dbo.AQ22ANCES_RESULTS'

# which column to use/create 
RESPONSE_COLUMN = 'aq22_cleaned'

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# amount of synthetic data to produce
N_SYNTHETIC_MIXED = 50_000

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Read from Database

In [ ]:
# Read in all data required to build model
connection = connect.create_connection(CRED_PATH)

# actual responses
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

# codes to match
code_df = connect.fetch_table(CODE_TABLE, connection)

# closed respones (for multi response frequencies)
df_closed = connect.fetch_table(CLOSED_TABLE, connection)

In [ ]:
# Reshaping the dataframe
df_melted = df_open.melt(id_vars=['id', 'cycle'], value_vars=[f'q22ances{i}' for i in range(1, 6)])
df_melted['aq22ances'] = df_open.melt(id_vars=['id', 'cycle'], value_vars=[f'aq22ances{i}' for i in range(1, 6)])['value']

# Keeping the columns of interest
df_reshaped = df_melted[['id', 'cycle', 'value', 'aq22ances']]
df_reshaped.columns = ['id', 'cycle', 'q22ances', 'aq22ances']

# Filtering rows where q22ances and aq22ances are not None
df_reshaped = df_reshaped[df_reshaped['q22ances'].notna() & df_reshaped['aq22ances'].notna()]

df_open = df_reshaped.sort_values('id')

In [ ]:
df_open[df_open.cycle == 1].head()

## Preprocess Data

* Use only cycle 1 for training.
* Use the cleaned column.
* Correct spelling where possible (takes longer time to run).
* Create input and output tables for model (input table takes a while to produce)

In [ ]:
# for training purposes, open responses should only be those from cycle 1
df = df_open[df_open.cycle == 1].reset_index(drop=True)
df.head()

In [ ]:
# Use the cleaned column if available; otherwise, use the original column
df.loc[:, RESPONSE_COLUMN] = df.apply(
    lambda x: str(x.aq22ances).lower() if x.aq22ances is not None else np.nan, 
    axis=1
)

# Now only apply correct_spelling on non-null values
mask = df[RESPONSE_COLUMN].notnull()
df.loc[mask, RESPONSE_COLUMN] = df.loc[mask, RESPONSE_COLUMN].apply(preprocess.correct_spelling)

df.head()

In [ ]:
# get long form table of codes
code_df_long = preprocess.get_long_form_codes(code_df)
code_df_long.head(20)

In [ ]:
# build a training dataset for the model 
# first part of dataset: actual data
code_df_long['code'] = code_df_long['code'].astype(str)
df[RESPONSE_COLUMN] = df[RESPONSE_COLUMN].astype(str)

# INPUTS TO MODEL
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)

input_df = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
display(input_df.head())

# OUTPUTS OF MODEL
# converts the coded columns into wide form 1/0 binary responses for every option 
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long)
output_df.head()

## Create Synthetic Data

* To augment our training data, produce synthetic data from the available phrases in the code list. 
* Both singular phrases and multi-response phrases are produced.
* The multi-response phrases are randomly generated according to weights associated with the non-written responses. 

In [ ]:
# create synthetic data
# this section will create synthetic data that matches a single category based on available phrases 
input_columns = list(input_df.columns)
output_columns = list(output_df.columns)

extra_input_df, extra_output_df = synthetic.create_single_phrase_synthetic(
    output_df, 
    input_columns,
    output_columns,
    code_df_long
)

In [ ]:
# create synthetic data
# this section will create synthetic data that matches multiple categories
mixed_input_df, mixed_output_df = synthetic.create_multi_phrase_synthetic(
    output_df,
    df_closed,
    input_columns,
    output_columns,
    code_df_long,
    N_SYNTHETIC_MIXED
)

In [ ]:
# Concatenate with existing data
final_input_df = pd.concat([input_df, extra_input_df, mixed_input_df], ignore_index=True).drop('response', axis=1).astype(int)
final_output_df = pd.concat([output_df, extra_output_df, mixed_output_df], ignore_index=True).drop('response', axis=1).astype(int)

## Train Model

To deal with the fact we have many possible categorical outputs, use a simple random forest model that will handle multiple outputs better than other models.

* After training, save model. 

In [ ]:
clf = model.create_model(final_input_df, final_output_df)

In [ ]:
model.save_model(CRED_PATH, MODEL_BASE_PATH, clf, code_df_long)

In [ ]:
clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)

## Produce Model Results

* Look at some results by hand to see if it makes sense
* Create results for entire hand-coded dataset
* Send results back to databse

In [ ]:
sentence = 'chinese, japanese, korean'
model.list_classes(sentence, code_df_long, clf, spellcheck=True)

In [ ]:
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)

In [ ]:
# save back to database
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)

# for initial save of cycle 1, always replace. any subsequent inputs should be appended
connect.save_table(results_df, RESULTS_TABLE, engine, how='replace')